<a href="https://colab.research.google.com/github/PedroSoria/dog-vision/blob/main/dog_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from IPython.display import Image
import os
from sklearn.model_selection import train_test_split
from matplotlib.pyplot import imread

In [ ]:
print(tf.__version__)
print(hub.__version__)
physical_devices = tf.config.list_physical_devices()
print(physical_devices)

2.15.0
0.16.1
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [ ]:
# !unzip "drive/MyDrive/[006] Datasets/[001] \dog-breed-identification/dog-breed-identification.zip" -d"drive/MyDrive/"

Archive:  drive/MyDrive/[006] Datasets/[001] \dog-breed-identification/dog-breed-identification.zip
replace drive/MyDrive/labels.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
lables_csv = pd.read_csv("drive/MyDrive/labels.csv")
filename = ["data/train/" + fname for fname in lables_csv.id + ".jpg"]
len(filename)
lables = lables_csv.breed.to_numpy()
lables
unique_breeds = np.unique(lables)
boolean_lables = [lable == unique_breeds for lable in lables]
len(unique_breeds)
X = filename
y = boolean_lables

In [ ]:
NUM_IMAGE = 1000
IMAGE_SIZE = 224
BATCH_SIZE = 32

In [ ]:
from sklearn.model_selection import train_test_split

X_train, x_val, y_train, y_val = train_test_split(X[:NUM_IMAGE],
                                                   y[:NUM_IMAGE],
                                                   test_size=0.2,
                                                   random_state=42)

In [ ]:
# Define Image Size
def procces_image(image_path):
    """_summary_

    Args:
        image_path (_type_): _description_
    """
    # read in an image file
    image = tf.io.read_file(image_path)
    # Turn the image into numerical Tensor with 3 coulor channels (read, green, blue)
    image = tf.image.decode_jpeg(image, channels=3)
    # Convert the colour channel values from 0-255 to 1-1 values
    image = tf.image.convert_image_dtype(image, tf.float32)
    # Rezice image to (224, 224)
    image = tf.image.resize(image, size=[IMAGE_SIZE, IMAGE_SIZE])

    return image

In [ ]:
def get_image_lable(image_path, lable):
    """_summary_

    Args:
        image_path (_type_): _description_
        lable (_type_): _description_
    """
    image = procces_image(image_path)

    return image, lable

In [ ]:
def create_data_batches(X,
                        y=None,
                        buffer_size=BATCH_SIZE,
                        valid_data=False,
                        test_data=False):
    """_summary_

    Args:
        X (_type_): _description_
        y (_type_, optional): _description_. Defaults to None.
        batch_size (_type_, optional): _description_. Defaults to BATCH_SIZE.
        valid_data (bool, optional): _description_. Defaults to False.
        test_data (bool, optional): _description_. Defaults to False.
    """
    if test_data:
        print("creating test dasta batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X)))
        data_batch = data.map(procces_image).batch(BATCH_SIZE)

        return data_batch

    elif valid_data:
        print("creating validation dasta batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X),
                                                   tf.constant(y)))
        data_batch = data.map(get_image_lable).batch(BATCH_SIZE)

        return data_batch

    else:
        print("creating training data batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X),
                                                   tf.constant(y)))
        data = data.shuffle(buffer_size=len(X))
        data = data.map(get_image_lable)
        data_batch = data.batch(BATCH_SIZE)

        return data_batch

In [ ]:
train_data = create_data_batches(X_train, y_train)
val_data = create_data_batches(x_val, y_val, valid_data=True)

creating training data batches...
creating validation dasta batches...


In [ ]:
# Setup shape to the model
INPUT_SHAPE = [None, IMAGE_SIZE, IMAGE_SIZE, 3] # batc, heigth, width , coulor channer

# Setup output of model
OUTPT_SHAPE = len(unique_breeds)

MODEL_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4"

In [ ]:
def create_model(input_shape=INPUT_SHAPE,
                 output_shape=OUTPT_SHAPE,
                 model_url=MODEL_URL):

    # Setup model layers
    model = tf.keras.Sequential([
        hub.KerasLayer(MODEL_URL),
        tf.keras.layers.Dense(units=OUTPT_SHAPE,
                              activation="softmax")
    ])

    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=["accuracy"]
    )

    model.build(INPUT_SHAPE)

    return model

In [ ]:
model = create_model()
model.summary()

In [ ]:
# Creatin Callbacks
%load_ext tensorboard
import datetime

def create_tensorboard_callback ():
  logdir = os.path.join("drive/MyDrive/",
                        datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  return tf.keras.Callbacks.TensorBoard(logdir)